In [171]:
import os

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"] = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [172]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [173]:
!pip install -q pydantic_ai openai
!pip install pytz

In [192]:
from datetime import datetime
from pydantic_ai import Tool          
@Tool
def get_current_date() -> str:
    """Return the current date/time as an ISO-formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [193]:
from typing import Annotated, List
from pydantic_ai import Agent, Tool
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic import BaseModel, Field

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

class MeetingMetadata(BaseModel):
    participants: List[str] = Field(..., description="list of emails who are the participants for the meeting.")
    start_time: str = Field(..., description="A valid format of time string for the Starting point of time window")
    end_time: str = Field(..., description="A valid format of time string for the ending point of time window")
    meeting_duration: int = Field(..., description="Duration of the meeting in minutes.")

@Tool
def get_time_window(
    current_date: Annotated[str, "The string formatted timestamp of current date & time"],
    time_constraints: Annotated[str, "The exact date, day or time window where the call can be scheduled"],
    meeting_duration: Annotated[int, "how long the call/meeting is going to occur, unit in minutes"]
) -> MeetingMetadata:
    """Based on the current_date, time_contstraints, and meeting_duration determine the date range for the required meeting timeframe (start and end date),
        where a work week is defined as Monday to Friday only (excluding weekends).
        - "Next week" always refers to the next Monday–Friday period, not 7 days from today.
        - For example, if today is Wednesday, then next week starts from the coming Monday.

        Calculate the start and end timestamps based on the current date, time_constraint and meeting duration given, keeping in mind that it is suppose to be only in work weekdays.
        Get the current day from the input current_date and then understand the actual start date and end date.
        Return all the following:
            The current timestamp
            A valid format of time string (e.g. '2025-07-14 09:00:00', '2025-07-14T03:30:00Z') for the Starting point of timewindow
            A valid format of time string (e.g. '2025-07-14 09:00:00', '2025-07-14T03:30:00Z') for the ending point of time window
            The time_duration for the meeting as it was in the input
    """

In [194]:
class MeetingInfo(BaseModel):
    participants: str = Field(..., description="Comma-separated emails of all participants.")
    time_constraints: str = Field(..., description="Mentioned timing or date phrase in the email.")
    meeting_duration: int = Field(..., description="Duration of the meeting in minutes.")

@Tool
def extract_meeting_info(
    email: Annotated[str, "The raw email body containing meeting details."]
) -> MeetingInfo:
    """
        Yor are an Agent that helps in scheduling meetings.
        Your job is to extracts Email ID's and Meeting Duration.
        You should return :
        1. List of email id's of participants (comma-separated).
        2. Meeting duration in minutes.
        3. Time constraints (e.g., 'next week').
        If the List of email id's of participants are just names, then append @amd.com at the end and return. 
        Return as json with 'participants', 'time_constraints' & 'meeting_duration'.
        Stricty follow the instructions. Strictly return dict with participents email id's, time constraints & meeting duration in minutes only.
    """

In [199]:
@Tool
def retrive_calendar_events(user: Annotated[str, "The mail id of each individual user."], 
                            start: Annotated[str, "A valid format of time string (e.g., '2025-07-17T00:00:00+05:30') for the Starting point of timewindow"],
                            end: Annotated[str, "A valid format of time string (e.g., '2025-07-17T00:00:00+05:30') for the ending point of time window"]
                           ) -> List[{"StartTime": str, "EndTime": str, "NumAttendees" : int, "Attendees" : List[str], "Summary" : str}]:
    """
        Fetch Google Calendar events for a user between start and end times, the times must be in a valid iso datetime format like (e.g., '2025-07-17T00:00:00+05:30').
        Returns a list of dicts with StartTime, EndTime, Attendees, Summary.
    """
    events_list = []
    token_path = "../Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start, timeMax=end, singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [200]:
from dateutil import parser
from datetime import datetime
import pytz

@Tool
def parse_to_iso_datetime(date_str: str, timezone_str: str = 'Asia/Kolkata') -> str:
    """
    Converts any human-readable date-time string to ISO format with timezone offset.
    
    Args:
        date_str (str): Input date-time string (e.g., 'next Thursday', 'July 17, 2025').
        timezone_str (str): Timezone name (default: 'Asia/Kolkata').
        
    Returns:
        str: ISO formatted date-time string with offset (e.g., '2025-07-17T00:00:00+05:30').
    """
    # Parse to naive datetime
    dt_naive = parser.parse(date_str, fuzzy=True)

    # Attach timezone
    tz = pytz.timezone(timezone_str)
    dt_localized = tz.localize(dt_naive)

    # Return ISO string with offset
    return dt_localized.isoformat()

In [201]:
agent_model = OpenAIModel("Qwen3-8b", provider=provider)

agent = Agent(
    model=agent_model,
    tools=[extract_meeting_info, get_current_date, get_avl_time_window, convert_to_local_timezone, retrive_calendar_events],
    system_prompt = """
    You are an AI agent that assists with meeting scheduling.
        Your tools include:
            - extract_meeting_info(email)
            - get_current_time()
            - get_time_window()
            - parse_to_iso_datetime()
            - retrive_calendar_events()

        Your task is to:
            1. Parse the email
            2. Determine the current date
            3. Find out the required meeting timeframe using get_time_window
            4. Convert the requried timeframe to a proper time format
            5. Retrive the calendar events for the attendees and suggest a correct time for the meeting
    """
)

In [202]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output

email_text = """
Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project. 
Attendees: "usertwo.amd@gmail.com", "userthree.amd@gmail.com"
"""

await run_async(f"Suggest a time for meeting as per the following email based on both the attendees's availability:\n\n{email_text}")

"\n\nBased on the attendees' calendars, here's the analysis:\n\n**Attendees:**\n1. usertwo.amd@gmail.com\n2. userthree.amd@gmail.com\n\n**Key Findings:**\n- **Next Thursday (July 17, 2025)** is the requested day, but both attendees have conflicts:\n  - **usertwo** has an urgent meeting from 10:00-10:30 AM\n  - **userthree**'s calendar isn't checked in this analysis\n\n**Available Window:**\n- **July 14-18, 2025** (as calculated)\n- **Best time:** 9:00 AM - 9:30 AM on July 14 (before usertwo's Off Hours start at 6:00 PM)\n\n**Recommendation:**\nPropose **July 14, 2025 at 9:00 AM** (9:00-9:30 AM). Check userthree's calendar for confirmation. If they're available, this time works; otherwise, consider July 15 (before 9:00 AM)."